# Group work - Assessment 2

In this assignment, we will focus on salary prediction. The data set for this assignment includes information on job descriptions and salaries. Use this data set to see if you can predict the salary of a job posting (i.e., the `Salary` column in the data set) based on the job description. This is important, because this model can make a salary recommendation as soon as a job description is entered into a system.

## Description of Variables

The description of variables are provided in "Jobs - Data Dictionary.docx"

## Goal

Use the **jobs_alldata.csv** data set and build models to predict **salary**.

**Be careful: this is a REGRESSION task**

## Submission:

Please save and submit this Jupyter notebook file. The correctness of the code matters for your grade. **Readability and organization of your code is also important.** You may lose points for submitting unreadable/undecipherable code. Therefore, use markdown cells to create sections, and use comments where necessary.


## Recommended roles for group members:

**Section 1:** to be completed by both group members

**Section 2:** first three models to be completed by the first group member and checked by the second; last two models to be completed by the second group members and checked by the first group member.

**Discussion:** to be completed by both group members

**Important notes:**
- Both group members will get the same grade. Therefore, you should check the work of your group member. If they make a mistake, you will be responsible for that mistake too.
- Both group members must put in their fair share of effort. Otherwise, those who don't contribute to the assignment will not receive any grade.


# Section 1: (8 points in total)

## Data Prep (6 points)

In [266]:
import pandas as pd
import numpy as np
import os

In [267]:
# get csv files from current directory
csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
csv_files

['jobs_alldata.csv']

In [268]:
# read csv files
df = pd.read_csv(csv_files[0])
df.head()

,Salary,Job Description,Location,Min_years_exp,Technical,Comm,Travel
0,67206,Civil Service Title: Regional Director Mental ...,Remote,5,2,3,0
1,88313,The New York City Comptrollerâ€™s Office Burea...,Remote,5,2,4,10-15
2,81315,With minimal supervision from the Deputy Commi...,East campus,5,3,3,5-10
3,76426,OPEN TO CURRENT BUSINESS PROMOTION COORDINATOR...,East campus,1,1,3,0
4,55675,Only candidates who are permanent in the Princ...,Southeast campus,1,1,3,5-10


In [269]:
# make column names lowercase
df.columns = map(str.lower, df.columns)
df.head()

,salary,job description,location,min_years_exp,technical,comm,travel
0,67206,Civil Service Title: Regional Director Mental ...,Remote,5,2,3,0
1,88313,The New York City Comptrollerâ€™s Office Burea...,Remote,5,2,4,10-15
2,81315,With minimal supervision from the Deputy Commi...,East campus,5,3,3,5-10
3,76426,OPEN TO CURRENT BUSINESS PROMOTION COORDINATOR...,East campus,1,1,3,0
4,55675,Only candidates who are permanent in the Princ...,Southeast campus,1,1,3,5-10


In [270]:
from sklearn.model_selection import train_test_split

# define train and test sets from data
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('salary', axis=1), df['salary'], test_size=0.2, random_state=0)


In [271]:
# check for null values
print(X_train.isnull().sum())

job description    0
location           0
min_years_exp      0
technical          0
comm               0
travel             0
dtype: int64


In [272]:
print(X_test.isnull().sum())

job description    0
location           0
min_years_exp      0
technical          0
comm               0
travel             0
dtype: int64


In [273]:
# print data types
print(X_train.dtypes)

job description    object
location           object
min_years_exp       int64
technical           int64
comm                int64
travel             object
dtype: object


In [274]:
# create a new variabel only containing numeric data
X_train_numeric = X_train.select_dtypes(include=[np.number]).columns.to_list()

# create a new variabel only containing categorical data
X_train_categorical = X_train.select_dtypes(include=[np.object])
X_train_cat = []
X_train_binary = []
# loop through the columns of the categorical data
for col in X_train_categorical:
    # check unique values per row
    x = len(X_train[col].unique())
    # check if there are more than 2 unique values
    if x > 2:
        X_train_cat.append(col)
    else:
        X_train_binary.append(col)

print(X_train_cat)
print(X_train_binary)
print(X_train_numeric)


['job description', 'location', 'travel']
[]
['min_years_exp', 'technical', 'comm']


C:\Users\nicol\AppData\Local\Temp\ipykernel_14256\1071052424.py:5: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train_categorical = X_train.select_dtypes(include=[np.object])


In [275]:
X_train_numeric

['min_years_exp', 'technical', 'comm']

In [276]:
X_train.head()

,job description,location,min_years_exp,technical,comm,travel
459,**THIS IS A TEMPORARY POSITION** In January 20...,Southeast campus,1,1,3,0
2006,"DoITT provides for the sustained, efficient an...",HQ,5,5,1,0
951,"Within HPDâ€™s Office of Legal Affairs, the Co...",Southeast campus,1,2,3,0
1598,Only candidates who are permanent in the Admin...,HQ,4,3,4,0
1458,Job Description 1. Oversee daily maintenance...,HQ,4,3,3,0


## Importing Necessary Decision Tree Libraries

In [277]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

### Feature Engineeering: Defining New Column based on the average from our numerical data (Min years of experience, Technical, Comm)

In [278]:
# define new column 'Combined_Aptitude' by averaging 'Min_years_exp' , 'Technical' and 'Comm'
def combine_aptitude(df):
    # copy df
    df_copy = df.copy()
    # create new column 'Combined_Aptitude'
    df_copy['combined_Aptitude'] = ((
        df_copy['min_years_exp'] + df_copy['technical'] + df_copy['comm']) / 3).fillna(0)

    df_copy['combined_Aptitude'].replace(np.inf, 1, inplace=True)
    # return new column
    return df_copy[['combined_Aptitude']]

# assign
feat_eng_cols = ['min_years_exp', 'technical', 'comm']
# X_train['Combined_Aptitude'] = combine_aptitude(X_train)
# X_test['Combined_Aptitude'] = combine_aptitude(X_test)

In [279]:
# X_train['combined_Aptitude'] = combine_aptitude(X_train)

In [280]:
X_train.head()

,job description,location,min_years_exp,technical,comm,travel
459,**THIS IS A TEMPORARY POSITION** In January 20...,Southeast campus,1,1,3,0
2006,"DoITT provides for the sustained, efficient an...",HQ,5,5,1,0
951,"Within HPDâ€™s Office of Legal Affairs, the Co...",Southeast campus,1,2,3,0
1598,Only candidates who are permanent in the Admin...,HQ,4,3,4,0
1458,Job Description 1. Oversee daily maintenance...,HQ,4,3,3,0


## Feature Engineering (1 points)

Create one NEW feature from existing data. You either transform a single variable, or create a new variable from existing ones. 

Grading: 
- 0.5 points for creating the new feature correctly
- 0.5 points for the justification of the new feature (i.e., why did you create this new feature)

In [281]:
feat_eng_cols

['min_years_exp', 'technical', 'comm']

### Why this feature?
We created this feature to aggregate all the numerical values and averaging them to have a better idea of the overall strengths of a candidate

## Find the Baseline (1 point)

In [282]:
from sklearn.metrics import mean_squared_error
# find the mean squared error of the model
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


In [283]:
print(X_train_numeric)
print(X_train_cat)
print(X_train_binary)
print(feat_eng_cols)

['min_years_exp', 'technical', 'comm']
['job description', 'location', 'travel']
[]
['min_years_exp', 'technical', 'comm']


In [284]:
# Initialize the Pipeline
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

categorial_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# bin_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent'))
# ])

my_new_col = Pipeline(steps=[('my_new_col', FunctionTransformer(combine_aptitude)),
                             ('scaler', StandardScaler())])


In [285]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, X_train_numeric),
        ('cat', categorial_pipeline, X_train_cat),
        ('my_new_col', my_new_col, feat_eng_cols)
    ], remainder='passthrough')

In [286]:
# fit and transform the data
X_train_transformed = preprocessor.fit_transform(X_train)

In [287]:
X_train_transformed

<1930x1309 sparse matrix of type '<class 'numpy.float64'>'
	with 13510 stored elements in Compressed Sparse Row format>

In [288]:
X_train_transformed.shape

(1930, 1309)

## Baseline Calculations (from RMSE definition way above)

In [289]:
# find the mean value of y_train
y_train_mean = y_train.mean()
y_train_mean

77804.90984455959

In [290]:
# prdict the salary based of the mean value of y_train
y_train_mean_pred = np.repeat(y_train_mean, len(y_test))
y_train_mean_pred

array([77804.90984456, 77804.90984456, 77804.90984456, 77804.90984456,
       77804.90984456, 77804.90984456, 77804.90984456, 77804.90984456,
       77804.90984456, 77804.90984456, 77804.90984456, 77804.90984456,
       77804.90984456, 77804.90984456, 77804.90984456, 77804.90984456,
       77804.90984456, 77804.90984456, 77804.90984456, 77804.90984456,
       77804.90984456, 77804.90984456, 77804.90984456, 77804.90984456,
       77804.90984456, 77804.90984456, 77804.90984456, 77804.90984456,
       77804.90984456, 77804.90984456, 77804.90984456, 77804.90984456,
       77804.90984456, 77804.90984456, 77804.90984456, 77804.90984456,
       77804.90984456, 77804.90984456, 77804.90984456, 77804.90984456,
       77804.90984456, 77804.90984456, 77804.90984456, 77804.90984456,
       77804.90984456, 77804.90984456, 77804.90984456, 77804.90984456,
       77804.90984456, 77804.90984456, 77804.90984456, 77804.90984456,
       77804.90984456, 77804.90984456, 77804.90984456, 77804.90984456,
      

In [297]:
# use rmse function to find the mean squared error
baseline_rmse = rmse(y_test, y_train_mean_pred)
baseline_rmse

29309.514702510158

In [300]:
y_train.head()

459      64498
2006    108566
951      76013
1598    147023
1458     66363
Name: salary, dtype: int64

# Section 2: (7 points in total)

Build the following models:


## Decision Tree: (1 point)

In [301]:
from sklearn.tree import DecisionTreeRegressor

In [319]:
tree_reg = DecisionTreeRegressor(random_state=0, min_samples_leaf=100)
tree_reg.fit(X_train_transformed, y_train)

DecisionTreeRegressor(min_samples_leaf=100, random_state=0)

In [320]:
# Train RMSE
train_pred = tree_reg.predict(X_train_transformed)
train_rmse = rmse(y_train, train_pred)
f'Train RMSE: {train_rmse:.2f}'

'Train RMSE: 28866.27'

In [321]:
# Test RMSE
test_pred = tree_reg.predict(preprocessor.transform(X_test))
test_rmse = rmse(y_test, test_pred)
f'Test RMSE: {test_rmse:.2f}'

'Test RMSE: 29617.47'

## Voting regressor (2 points):

The voting regressor should have at least 3 individual models

## A Boosting model: (1 point)

Build either an Adaboost or a GradientBoost model

## Neural network: (1 point)

## Grid search (2 points)

Perform either a full or randomized grid search on any model you want. There has to be at least two parameters for the search. 

In [323]:
from sklearn.model_selection import RandomizedSearchCV


In [330]:
param_grid = {
    'min_samples_leaf': np.arange(100, 5000, 100),
    'max_depth': np.arange(2, 50, 2),
    'max_features': ['auto', 'sqrt', 'log2']
}

tree_reg = DecisionTreeRegressor(random_state=0)

In [331]:
grid_search = RandomizedSearchCV(tree_reg, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, return_train_score=True)

grid_search.fit(X_train_transformed, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=0),
                   param_distributions={'max_depth': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34,
       36, 38, 40, 42, 44, 46, 48]),
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': array([ 100,  200,  300,  400,  500,  600,  700,  800,  900, 1000, 1100,
       1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200,
       2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300,
       3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400,
       4500, 4600, 4700, 4800, 4900])},
                   return_train_score=True, scoring='neg_mean_squared_error',
                   verbose=1)

In [332]:
cvres = grid_search.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

29198.187476899 {'min_samples_leaf': 3200, 'max_features': 'log2', 'max_depth': 6}
29198.187476899 {'min_samples_leaf': 2900, 'max_features': 'log2', 'max_depth': 40}
29198.187476899 {'min_samples_leaf': 3200, 'max_features': 'auto', 'max_depth': 32}
29198.187476899 {'min_samples_leaf': 4200, 'max_features': 'log2', 'max_depth': 40}
29194.227639260655 {'min_samples_leaf': 300, 'max_features': 'log2', 'max_depth': 4}
29198.187476899 {'min_samples_leaf': 1700, 'max_features': 'auto', 'max_depth': 28}
29198.187476899 {'min_samples_leaf': 3300, 'max_features': 'auto', 'max_depth': 34}
29198.187476899 {'min_samples_leaf': 1300, 'max_features': 'log2', 'max_depth': 30}
29198.187476899 {'min_samples_leaf': 1100, 'max_features': 'log2', 'max_depth': 30}
29198.187476899 {'min_samples_leaf': 2900, 'max_features': 'sqrt', 'max_depth': 6}


In [333]:
grid_search.best_params_

{'min_samples_leaf': 300, 'max_features': 'log2', 'max_depth': 4}

In [334]:
grid_search.best_estimator_

DecisionTreeRegressor(max_depth=4, max_features='log2', min_samples_leaf=300,
                      random_state=0)

In [335]:
# Train RMSE
train_pred = grid_search.best_estimator_.predict(X_train_transformed)
train_rmse = rmse(y_train, train_pred)
f'Train RMSE: {train_rmse:.2f}'

'Train RMSE: 29125.54'

In [336]:
# Test RMSE
test_pred = grid_search.best_estimator_.predict(preprocessor.transform(X_test))
test_rmse = rmse(y_test, test_pred)
f'Test RMSE: {test_rmse:.2f}'

'Test RMSE: 29334.88'

# Discussion (5 points in total)


## List the train and test values of each model you built (2 points)

## Which model performs the best and why? (0.5 points) 
## How does it compare to baseline? (0.5 points)

Hint: The best model is the one that has the highest TEST score (regardless of any of the training values). If you select your model based on TRAIN values, you will lose points.

## Is there any evidence of overfitting in the best model, why or why not? If there is, what did you do about it? (1 point)

## Is there any overfitting in the other models (besides the best model), why or why not? If there is, what did you do about it? (1 point)